In [1]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

# Definieer gewichten en genormaliseerde amplituden
omega = [1.5, 0.5, -0.5]
lam = sum(abs(w) for w in omega)            # lambda = 2.5
alpha = [np.sqrt(abs(w)/lam) for w in omega]  # amplitudes voor |00>,|01>,|10>
alpha.append(0)                            # pad amplitude voor |11>
alpha = np.array(alpha, dtype=complex)
alpha = alpha / np.linalg.norm(alpha)      # normaliseer (zou al genormaliseerd moeten zijn)

# Bouw PREP subcircuit
anc = QuantumRegister(2, name="anc")
prep_circ = QuantumCircuit(anc, name="PREP")
prep_circ.prepare_state(alpha, anc)  # bereidt ancilla-toestand volgens alpha
prep_gate = prep_circ.to_gate(label="PREP")


In [2]:
prep_circ.draw()    
prep_circ.save_statevector()

In [3]:
from qiskit_aer import AerSimulator
# Set up the quantum simulator
backend = AerSimulator(method='statevector', shots=1024)
#Execute the circuit on the simulator
#job = backend.run(qpe_circ, shots=1024)
job = backend.run(prep_circ.decompose(reps=6), shots=1024)
state_prep = job.result().get_statevector(prep_circ)
print("Prepared state (ancilla):", state_prep)



Prepared state (ancilla): Statevector([7.74596669e-01+4.74303666e-17j,
             4.47213595e-01+1.36919675e-16j,
             4.47213595e-01+1.67761203e-16j,
             5.55111512e-17-7.11455499e-17j],
            dims=(2, 2))


In [4]:
from qiskit.circuit.library import XGate, IGate

# Definieer één-qubit gates voor I, X en -Z
id_gate = IGate()                   # identiteitspoort (I)
x_gate = XGate()                    # Pauli-X poort
# Bouw -Z gate via conjugatie X Z X
negz_circ = QuantumCircuit(1, name="-Z")
negz_circ.x(0); negz_circ.z(0); negz_circ.x(0)
negz_gate = negz_circ.to_gate(label="-Z")

# Maak lijst van unitaire gates [I, X, -Z]
U_list = [id_gate, x_gate, negz_gate]

# Bouw SELECT subcircuit: toepassen U_i afhankelijk van ancilla |i>
sys = QuantumRegister(1, name="sys")
select_circ = QuantumCircuit(anc, sys, name="SELECT")
m = anc.size  # =2 control qubits
for i, U_gate in enumerate(U_list):
    ctrl_state = format(i, f"0{m}b")[::-1]     # binaire representatie met 2 bits
    controlled_U = U_gate.control(m, ctrl_state=ctrl_state)
    select_circ.append(controlled_U, anc[:] + sys[:])
    print(f"U_{i} = {U_gate} met controle toestand {ctrl_state}")
select_gate = select_circ.to_gate(label="SELECT")


U_0 = Instruction(name='id', num_qubits=1, num_clbits=0, params=[]) met controle toestand 00
U_1 = Instruction(name='x', num_qubits=1, num_clbits=0, params=[]) met controle toestand 10
U_2 = Instruction(name='-Z', num_qubits=1, num_clbits=0, params=[]) met controle toestand 01


In [5]:
print(select_gate)

Instruction(name='SELECT', num_qubits=3, num_clbits=0, params=[])


In [6]:
select_circ.draw()

anc_0: ──o─────o────■───
         │     │    │   
anc_1: ──o─────■────o───
       ┌─┴──┐┌─┴─┐┌─┴──┐
  sys: ┤ Id ├┤ X ├┤ -Z ├
       └────┘└───┘└────┘

In [7]:

from qiskit import ClassicalRegister
c_reg = ClassicalRegister(4, name="c")

blockenc_circ = QuantumCircuit(anc, sys, name="U_block")
#blockenc_circ.x(anc[0])

blockenc_circ.append(select_gate, anc[:] + sys[:])

In [8]:
blockenc_circ.draw()

┌─────────┐
anc_0: ┤0        ├
       │         │
anc_1: ┤1 SELECT ├
       │         │
  sys: ┤2        ├
       └─────────┘

In [9]:
blockenc_circ.save_statevector()

In [10]:
from qiskit_aer import AerSimulator
# Set up the quantum simulator
backend = AerSimulator(method='statevector', shots=1024)
#Execute the circuit on the simulator
#job = backend.run(qpe_circ, shots=1024)
job = backend.run(blockenc_circ.decompose(reps=6), shots=1024)
state_prep = job.result().get_statevector(blockenc_circ)
print("Prepared state (ancilla):", state_prep)



Prepared state (ancilla): Statevector([ 1.00000000e+00-6.71997040e-16j,
              1.27639409e-33-7.88850967e-33j,
              1.97653216e-32+1.16379091e-32j,
             -1.56313193e-49+1.42523282e-48j,
              3.33066907e-16+2.35513869e-16j,
             -7.43936991e-33+1.72553385e-33j,
             -1.10495576e-32+5.47493331e-33j,
             -5.03265091e-64-4.59169004e-49j],
            dims=(2, 2, 2))


In [11]:
blockenc_circ.measure_all()

In [12]:
blockenc_circ.draw()

┌─────────┐ statevector  ░ ┌─┐      
 anc_0: ┤0        ├──────░───────░─┤M├──────
        │         │      ░       ░ └╥┘┌─┐   
 anc_1: ┤1 SELECT ├──────░───────░──╫─┤M├───
        │         │      ░       ░  ║ └╥┘┌─┐
   sys: ┤2        ├──────░───────░──╫──╫─┤M├
        └─────────┘      ░       ░  ║  ║ └╥┘
meas: 3/════════════════════════════╩══╩══╩═
                                    0  1  2

In [13]:
from qiskit_aer import AerSimulator
# Set up the quantum simulator
backend = AerSimulator()

#Execute the circuit on the simulator
#job = backend.run(qpe_circ, shots=1024)
job = backend.run(blockenc_circ.decompose(reps=6),shots=1024)
sim_result = job.result()
counts = sim_result.get_counts()

In [14]:
counts

{'000': 1024}

In [15]:
# Combineer PREP, SELECT, PREP† tot block-encoding unitary U
blockenc_circ = QuantumCircuit(anc, sys, name="U_block")
blockenc_circ.append(prep_gate, anc[:])
blockenc_circ.append(select_gate, anc[:] + sys[:])
blockenc_circ.append(prep_gate.inverse(), anc[:])
U_block = blockenc_circ.to_gate(label="U")  # gate voor de gehele block-encode


In [16]:
blockenc_circ.draw()

┌───────┐┌─────────┐┌──────────┐
anc_0: ┤0      ├┤0        ├┤0         ├
       │  PREP ││         ││  PREP_dg │
anc_1: ┤1      ├┤1 SELECT ├┤1         ├
       └───────┘│         │└──────────┘
  sys: ─────────┤2        ├────────────
                └─────────┘

In [17]:
blockenc_circ = blockenc_circ.decompose(reps=6)

In [18]:
# Bouw het quantum walk operator W = R * U
W_circ = QuantumCircuit(anc, sys, name="W")
# Eerst U toepassen:
W_circ.append(U_block, anc[:] + sys[:])
# Vervolgens reflectie R op ancilla:
W_circ.x(anc[0]); W_circ.x(anc[1])
W_circ.cz(anc[0], anc[1])            # faseflip op |11> (komt overeen met |00> origineel)
W_circ.x(anc[0]); W_circ.x(anc[1])
W_gate = W_circ.to_gate(label="W")


In [19]:
from qiskit.circuit.library import QFT

# Maak registers voor QPE, ancilla, en systeem
qpe = QuantumRegister(4, name="qpe")
anc_final = QuantumRegister(2, name="anc")   # ancilla (2 qubits)
sys_final = QuantumRegister(1, name="sys")
# klassiek register voor uitlezen fasebits
from qiskit import ClassicalRegister
c_reg = ClassicalRegister(4, name="c")

qpe_circ = QuantumCircuit(qpe, anc_final, sys_final, c_reg, name="QPE")
# 1. Initialiseer ancilla en systeem
qpe_circ.append(prep_gate, anc_final[:])          # voorbereid ancilla zoals eerder (alternatief: begin in |00> en laat PREP deel van U doen)
#qpe_circ.initialize([1,0], sys_final)             # bijvoorbeeld |0> als starttoestand systeem
# Vervang de standaard initialisatie van de systemqubit
#eigvec = np.array([], dtype=complex)  # |0> als starttoestand systeem
# Bereken de ongewenste vector (proportioneel)
#eigvec = np.array([1, 1 - np.sqrt(2)], dtype=complex)
eigvec = np.array([1, 0])
# Normaliseer de vector
#eigvec = eigvec / np.linalg.norm(eigvec)
eigvec = eigvec / np.sqrt(np.sum(np.abs(eigvec)**2))

qpe_circ.initialize(eigvec, sys_final)
#v_{\text{norm}} \approx \begin{pmatrix}0.9238795 \\ -0.3826834\end{pmatrix},

# 2. Hadamards op alle QPE qubits
qpe_circ.h(qpe)

# 3. Gecontroleerde W^1, W^2, W^4 toepassen
# Bereid powers van W als gates (W, W^2, W^4)
from qiskit.quantum_info import Operator
W_op = Operator(W_gate)
W2_op = W_op.power(2);  W4_op = W_op.power(4); W8_op = W_op.power(8)
W2_gate = W2_op.to_instruction()
W4_gate = W4_op.to_instruction()
W8_gate = W8_op.to_instruction()

# Controleer met QPE qubits
qpe_circ.append(W_gate.control(1), [qpe[0]] + anc_final[:] + sys_final[:])
qpe_circ.append(W2_gate.control(1), [qpe[1]] + anc_final[:] + sys_final[:])
qpe_circ.append(W4_gate.control(1), [qpe[2]] + anc_final[:] + sys_final[:])
qpe_circ.append(W8_gate.control(1), [qpe[3]] + anc_final[:] + sys_final[:])

# 4. Inverse Quantum Fourier Transform op QPE-register
qft_inv = QFT(num_qubits=4, inverse=True).to_gate(label="QFT†")
qpe_circ.append(qft_inv, qpe)

# 5. Meet het fase-register
qpe_circ.measure(qpe, c_reg)


In [20]:
eigvec

array([1., 0.])

In [21]:
print(qpe_circ)

              ┌───┐                                                 ┌───────┐»
qpe_0: ───────┤ H ├─────────■───────────────────────────────────────┤0      ├»
              ├───┤         │                                       │       │»
qpe_1: ───────┤ H ├─────────┼────────■──────────────────────────────┤1      ├»
              ├───┤         │        │                              │  QFT† │»
qpe_2: ───────┤ H ├─────────┼────────┼───────────■──────────────────┤2      ├»
              ├───┤         │        │           │                  │       │»
qpe_3: ───────┤ H ├─────────┼────────┼───────────┼───────────■──────┤3      ├»
            ┌─┴───┴─┐     ┌─┴──┐┌────┴─────┐┌────┴─────┐┌────┴─────┐└───────┘»
anc_0: ─────┤0      ├─────┤0   ├┤0         ├┤0         ├┤0         ├─────────»
            │  PREP │     │    ││          ││          ││          │         »
anc_1: ─────┤1      ├─────┤1 W ├┤1 Unitary ├┤1 Unitary ├┤1 Unitary ├─────────»
       ┌────┴───────┴────┐│    ││          ││       

In [22]:
from qiskit_aer import AerSimulator
# Set up the quantum simulator
backend = AerSimulator()
#Execute the circuit on the simulator
#job = backend.run(qpe_circ, shots=1024)
job = backend.run(qpe_circ.decompose(reps=6),shots=1024)
sim_result = job.result()
counts = sim_result.get_counts()

In [23]:
counts

{'0001': 1,
 '1101': 2,
 '0100': 5,
 '1011': 508,
 '0111': 38,
 '0011': 3,
 '0000': 148,
 '0010': 1,
 '1100': 37,
 '1001': 146,
 '0101': 83,
 '1110': 3,
 '1010': 31,
 '1111': 2,
 '1000': 9,
 '0110': 7}

In [24]:
t = 1
phase_bits = max(counts, key=counts.get) # take the most often obtaned result

phase = 0
for index, bit in enumerate((phase_bits)):
    phase += int(bit) / 2**(index + 1)
    print(phase)

lam = 2.5
# Bereken de eigenwaarde van de Hamiltoniaan
# met behulp van de gemeten fase
E = 0
theta = 2*np.pi*phase
if theta < np.pi/2:
    E =  lam * np.cos(theta)
else:
    E = -lam * np.cos(theta)



print("Estimated eigenvalue of the Hamiltonian: {}".format(E))

0.5
0.5
0.625
0.6875
Estimated eigenvalue of the Hamiltonian: 0.9567085809127258


In [25]:
phase_bits

'1011'

In [26]:
phase

0.6875

In [27]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import XGate, IGate
from qiskit.quantum_info import Operator
import numpy as np

# --- PREP (ancilla‑superpositie) ---
omega = [1.5, 0.5, -0.5]; lam = 2.5
alpha = [np.sqrt(abs(w)/lam) for w in omega] + [0]      # |00>,|01>,|10>,|11>
anc   = QuantumRegister(2,'anc')
prep  = QuantumCircuit(anc, name='PREP')
prep.prepare_state(alpha, anc)
prep_gate = prep.to_gate()

# --- SELECT (conditioneel I,X,-Z op systeem) ---
sys = QuantumRegister(1,'sys')
idg, xg = IGate(), XGate()
negz = QuantumCircuit(1); negz.x(0); negz.z(0); negz.x(0)
select = QuantumCircuit(anc, sys, name='SELECT')
for i,U in enumerate([idg,xg,negz.to_gate(label='-Z')]):
    #select.append(U.control(2, ctrl_state=f'{i:02b}'), anc[:] + sys[:])
    # reverse the two‐bit string so bit0→anc[0], bit1→anc[1]
    cs = f'{i:02b}'[::-1]
    select.append(U.control(2, ctrl_state=cs), anc[:] + sys[:])
select_gate = select.to_gate()

# --- U = PREP · SELECT · PREP† ---
Ucirc = QuantumCircuit(anc, sys)
Ucirc.append(prep_gate, anc)
Ucirc.append(select_gate, anc[:] + sys[:])
Ucirc.append(prep_gate.inverse(), anc)
U = Operator(Ucirc).data        # 8×8‑matrix (2 ancilla‑qubits + 1 systeem‑qubit)


In [28]:
 # pick out the anc=|00> subspace (b0=0,b1=0) → indices 0 and 4
idx = [0, 4]
top_left = U[np.ix_(idx, idx)]

In [29]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])
H_expected = (1.5*I - 0.5*Z + 0.5*X)/lam


In [30]:
print("Extracted:\n", np.round(top_left,10))
print("Expected :\n", np.round(H_expected,10))
print("Overeenkomst?:", np.allclose(top_left, H_expected, atol=1e-8))


Extracted:
 [[0.4-0.j 0.2-0.j]
 [0.2-0.j 0.8-0.j]]
Expected :
 [[0.4 0.2]
 [0.2 0.8]]
Overeenkomst?: True
